In [55]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [56]:
df = pd.read_csv('./data/good_data/fraudTrain.csv')

In [64]:
# Drop Unecessary Data
# First, last name, job, dob unecessary PII
# Get rid of labels for unsupervised learning (is_fraud)
# CC_Num, trans_num unique identifier so not necessary
# Merchant: Already have location so not necessary
# Street, city, and zip not necessary since we have lat and lng

unecessary_columns = [
    'first', 
    'last', 
    'is_fraud', 
    'cc_num', 
    'merchant', 
    'street', 
    'city', 
    'zip', 
    'city_pop',
    'job',
    'dob',
    'trans_num',
    'trans_date_trans_time',
    'unix_time',
    'state',
    'Unnamed: 0'
]
unsupervised_df = df.drop(columns=unecessary_columns)

le = LabelEncoder()
unsupervised_df['category'] = le.fit_transform(unsupervised_df['category'])
unsupervised_df['gender'] = le.fit_transform(unsupervised_df['gender'])

,category,amt,gender,lat,long,merch_lat,merch_long
0,8,4.97,0,36.0788,-81.1781,36.011293,-82.048315
1,4,107.23,0,48.8878,-118.2105,49.159047,-118.186462
2,0,220.11,1,42.1808,-112.2620,43.150704,-112.154481
3,2,45.00,1,46.2306,-112.1138,47.034331,-112.561071
4,9,41.96,1,38.4207,-79.4629,38.674999,-78.632459
...,...,...,...,...,...,...,...
1296670,0,15.56,1,37.7175,-112.4777,36.841266,-111.690765
1296671,1,51.70,1,39.2667,-77.5101,38.906881,-78.246528
1296672,1,105.93,1,32.9396,-105.8189,33.619513,-105.130529
1296673,1,74.90,1,43.3526,-102.5411,42.788940,-103.241160
